# **ENOE $^N$: Un Análisis Geoespacial de la Informalidad en México (2022)**

---

## **Metodología para la medición de Informalidad**

Con base en *Conociendo la base de datos de la ENOE. Datos ajustados a proyecciones de población 2010. (2020)*, INEGI.


> **Resumen de la base de datos de la ENOE**: Por cada vivienda existe uno o más registros de hogares, para cada hogar con entrevista completa (más
adelante se defi ne entrevista completa) se cuenta con el registro de las características sociodemográfi cas
de cada residente, es decir, cada hogar tiene a sus residentes en la tabla de sociodemográfico (SDEM)
y para cada residente con edad de 12 años cumplidos ó más se cuenta con un registro del Cuestionario
de Ocupación y Empleo, el cual se almacena en las tablas COE1 (de batería 1 a 5) y COE2 (de batería
6 a la última).

Basándose en lo anterior, se concluye que es posible identificar a los **residentes de un hogar** en la tabla de **caracteristicas sociodemográficas** (`SDEM`) dentro de la base de datos de la **ENOE**. Es importante señalar que cada **fila/registro** dentro de esta última, corresponde a las cáracteristicas de una única persona. Sin embargo, se puede conocer la **Frecuencia absoluta expandida** i.e. el número de personas que cuentan con las mismas mediante el **factor de expansión** en el **campo/columna** `FAC`.

> “El factor de expansión es el coeficiente que le otorga determinado peso en la muestra en función de su representatividad de otros tantos casos 
similares a él tomando en cuenta su estrato socioeconómico y lugar de residencia”.

$$ registro_i \times FAC = \text{Frecuencia absoluta expandida} \tag{1}$$

**Nota**: En la nueva base de datos **ENOE $^N$**, la variable `FAC` es reemplazada por `fac_tri` y `fac_mes`

En términos técnicos, es posible identificar a la población en el **sector informal** al filtrar por los registros que, en el campo de **Tipo de Unidad Económica 2** (`TUE2`) de la tabla `SDEM` cuenten con un valor de **5**.

```SQL

* Con esta sentencia se obtiene a la población económicamente activa.
SUM FAC FOR R_DEF=’00’ AND (C_RES=1 OR C_RES=3) AND (EDA>=’15’ AND EDA<=’98’) AND
CLASE1=1

* Para calcular los indicadores estratégicos e.g. Población en informalidad, se les aplica
* el pre-criterio siguiente:
R_DEF=’00’ AND (C_RES=’1’ OR C_RES=’3’) AND (EDA>=’15’ AND EDA<=’98’)

* Con esta se identificaría a la población en el sector informal
SELECT sum FAC FROM SDEM ... AND TUE2 = 5
```

Dando continuidad a la parte técnica, algo que se destaca es que, además del filtro de `TUE=5`, el cuál nos permite identificar personas dentro del **sector informal**, otras variables y criterios son empleados para encontrar a estos residentes: `R_DEF`, `C_RES` y `EDA`. El **INEGI** los describe como:

- > `R_DEF`: Existe un campo en la tabla HOGARES con el nombre “R_DEF” que almacena el resultado definitivo
de la entrevista del hogar, una ENTREVISTA COMPLETA se determina cuando en el campo “R_DEF”
almacena el valor ‘00’ (doble cero). Todo valor diferente a 00, es considerado como NO ENTREVISTA (ver documento de captación CARACTERISTICAS SOCIODEMOGRAFICAS en el apartado resultados de la entrevista).

- > `C_RES`:  En la tabla SOCIODEMOGRÁFICO existe el campo “C_RES”, que determina la condición de residencia
del ocupante del hogar. Si el valor que almacena este campo es 1, se trata de un residente habitual; si el
valor es 2, se trata de ausente definitivo; si el valor es 3, es un nuevo residente. Para mayor información
ver documento metodológico de la ENOE.

- > `EDA`:  En el campo “EDA”, se almacenan los años cumplidos. Si la edad del residente es de 12 años y más y
además tiene un 1 o un 3 en el campo “C_RES”, debe de existir un registro en la tabla CUESTIONARIO
DE OCUPACIÓN Y EMPLEO, si no cumple con estas características se termina su secuencia en la tabla
SOCIODEMOGRÁFICO.

Con base en lo anterior, se concluye que para que **un residente** sea considerado para calcular **indicadores estratégicos** es necesario que este: **1)** cuente con **entrevista completa** (`R_DEF = '00'`), **2)** sea **residente habitual** o **nuevo** (`C_RES` = '1' OR `C_RES` = '3'), y **3)** su edad se encuentre dentro de los **15** y **98** años (`EDA` >= '15' AND `EDA` <= '98').

Para calcular la **tasa de desocupación en el sector informal**, utilizado como un **acercamiento y comparativo** de la **informalidad** a nivel **entidad federativa** y **municipio**, se utiliza la formula definida por la **INEGI** dentro de su documento *Conociendo la base de datos de la ENOE (2020)*, (p.27):

**VIII.-Tasa de ocupación en el sector informal**

```SQL
Universo: R_DEF=00 y C_RES=1 ó 3 y edad=15-98
```

$$ TOSI = \frac{POSI}{PO} \times 100 \tag{2}$$

**Donde**:
- **POSI**: Población ocupada en el sector informal (`TUE2=5`)
- **PO**: Población ocupada (`Clase2 = 1`)

## **Fuentes**

- INEGI. (2020). *Cómo se hace la ENOE. Métodos y Procedimientos*. https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/702825190613.pdf
- INEGI. (2020). *Conociendo la base de datos de la ENOE. Datos ajustados a proyecciones de población 2010.*. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/con_basedatos_proy2010.pdf
- INEGI. (2021). *Encuesta Nacional de Ocupación y Empleo Nueva Edición (ENOE $^N$) Estructura de la base de datos*. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_321_fd_c_bas_amp.pdf 
- [TBC](https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/metodologias/ENOE/ENOE2014/informal_laboral/702825060459.pdf)

### **Bases de datos**
- Conjunto de datos + metadatos: https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/datosabiertos/2022/conjunto_de_datos_enoen_2022_1t_csv.zip
- Conjunto de datos simplificado (solo tablas en excel): https://www.inegi.org.mx/programas/enoe/15ymas/#Microdatos

### **Benchmark del 1er trimestre**
- Encuesta Nacional de Ocupación y Empleo (Nueva Edición) ENOE N. Resultados del primer trimestre de 2022. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_presentacion_ejecutiva_trim1_2022.pdf
- Comunicado de Prensa. Encuesta Nacional de Ocupación y Empleo, Nueva Edición. Primer Trimestre de 2022. https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_nota_tecnica_trim1_2022.pdf

# **TO DO**:

- Elaborar mapas para ubicar la informalidad en México
- Ubicar las localidades/municipios con mayor porcentaje de población en el sector informal
- Identificar las zonas rurales y urbanas con mayor porcentaje de población en el sector informal
- Analizar características socioeconómicas relevantes por región y presentarlas en los mapas
    - `SEX` Proporción hombre - mujer (poblacion total en localidad)
    - **calcular** Tasa de ocupación
    - **calcular** Tasa de ocupación informal
    - `EDA` Edad ¿cuál es la mediana y promedio de edad? la desviación estandar de la edad pude decirnos algo?
    - Año promedio de escolaridad `CS_P13_1`, `CS_P13_2`
    - `N_HIJ` Número de hijos nacidos
    - `E_CON` el estar casado tiene relacion con la informalidad?
    - `RAMA` sector económico. El sector con mayor proporcion tiene que ver con la informalidad?
        - `RAMA_EST_1` sector total (primario, secundario, ...)
    - `ANIOS_ESC:num` años de escolaridad
    - `HRSOCUP:num` horas trabajadas en la semana
    - `INGOCUP:num` Ingreso mensual
    - `ING_X_HRS`: Promedio de ingreso por hora trabajada

In [6]:
import pandas as pd # Manipulación de datos
import geopandas as gpd # Manipulación geospacial de datos
import matplotlib.pyplot as plt # 

import requests # Peticiones web
import os # Manipulacion del Sistema Operativo
import re # Regular expressions

from zipfile import ZipFile # Zipfile manipulacion

import warnings
warnings.filterwarnings('ignore')

Para facilitar el acceso a los archivos y el replicar este **jupyter notebook**, se descargan los datos de la ENOE $^N$ de forma automatizada.

In [7]:
# Define una lista con los links de los zipFiles que contienen las tablas de la base de datos
# de la enoen
enoen_links = [
    'https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim1_csv.zip',
    'https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim2_csv.zip',
    'https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim3_csv.zip',
    'https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim4_csv.zip'
]


# Itera sobre cada uno de lis links de los trimestres de la enoeN 2022 y descarga
# los archivos si aun no estan
enoe_n_path = 'data/enoe_n_2022/'
for trim, enoen_link in enumerate(enoen_links, start = 1):
    current_trim = f'enoe_n_2022_{trim}t'

    # Valida si no se encuentra la tabla de sociodemograficos del trimestre
    if not f'ENOEN_SDEMT{trim}22.csv' in os.listdir(enoe_n_path):
        print(f'[*] \'{current_trim}\' no fue encontrado. Descargando los archivos :)')

        # Haz un web request al INEGI para descargar la data
        enoen = requests.get(enoen_link)
        enoen_zip = f'{current_trim}.zip'

        # Abre un zipfile y escribe el contenido del web request
        open(enoen_zip, 'wb').write(enoen.content)

        # Extrae las tablas dentro del zipfile del trimstre
        with ZipFile(enoen_zip, 'r') as zObject:
            zObject.extractall(enoe_n_path)

        # Elimina el zip file, y solo quedate con las tablas
        os.remove(enoen_zip)

        print(f'[*] \'{enoen_zip}\' ha sido descargado y descomprimido en \'{enoe_n_path}\'\n')

    else:
        print(f'[*] {current_trim} ya fue descargado :)')

[*] enoe_n_2022_1t ya fue descargado :)
[*] enoe_n_2022_2t ya fue descargado :)
[*] enoe_n_2022_3t ya fue descargado :)
[*] enoe_n_2022_4t ya fue descargado :)


Extra los datos **geográficos** del **INEGI**

In [8]:
geo_data_path = 'data/mg_sep2019_integrado'

# Intenta listar el folder con la data geografica
try:
    os.listdir(geo_data_path)
    print('[*] La data geográfica ya esta descargada :)')
except:
    # Crea el folder
    os.mkdir(geo_data_path)
    print(f'[*] Creando el directorio \'{geo_data_path}\'')

    # Descarga los archivos
    print('[*] Descargando la data geográfica...')
    r = requests.get('http://internet.contenidos.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463776079/mg_sep2019_integrado.zip')
    open('mg_sep2019_integrado.zip', 'wb').write(r.content)
    
    # Extrae solo los datos del conjunto de datos para mun y ent
    with ZipFile('mg_sep2019_integrado.zip', 'r') as zipObject:
        filenames = [filename for filename in zipObject.namelist() if '00mun' in filename or '00ent' in filename]
        for filename in filenames:
            print(f'[*] Extrayendo \'{filename}\' en \'{geo_data_path}\'')
            zipObject.extract(filename, 'data/')
    
    # Una vez extraidos los archivos, elimina el zip file
    os.remove('mg_sep2019_integrado.zip')

[*] La data geográfica ya esta descargada :)


## **Importa los datos**

Para este trabajo, en esencia, se utilizará la tabla de **demográficos** de la base de de datos de la ENOE $^N$.

```python
# Genera una lista para guardar los dataframes de demograficos
sdemts = []

# Itera sobre cada uno de los archivos correspondientes a la tabla
# de demograficos
for file in [file for file in os.listdir(enoe_n_path) if 'SDEMT' in file]:
    print(f'[*] Leyendo \'{file}\'...')

    # Abre el archivo
    sdmet = pd.read_csv(os.path.join(enoe_n_path, file), encoding = 'latin_1')

    # Extrae el trimestre al que corresponde
    sdmet['trim'] = re.findall('[\d]', file)[0]

    # Guardalo en la lista de sdemts
    sdemts.append(sdmet)

# Concatena los dataframes en uno mismo
sdemts = pd.concat(sdemts, ignore_index=True)

# Imprime la información del dataframe
print(sdemts.info(), end = '\n'*2)

# Imprime el head del dataframe
sdemts.head()
```

In [28]:
# Importa el trimestre 1 para recon
df = pd.read_csv('data/enoe_n_2022/ENOEN_SDEMT122.csv', encoding = 'latin_1')

df.head()

,r_def,loc,mun,est,est_d_tri,est_d_men,ageb,t_loc_tri,t_loc_men,cd_a,...,scian,t_tra,emp_ppal,tue_ppal,trans_ppal,mh_fil2,mh_col,sec_ins,tipo,mes_cal
0,0,,2,10,122,,0,1,,1,...,6,1,2,2,0,3,2,2,1,96
1,0,,2,10,122,,0,1,,1,...,12,1,2,2,0,3,2,2,1,96
2,0,,2,10,122,,0,1,,1,...,12,1,2,2,0,3,6,4,1,96
3,0,,2,10,122,,0,1,,1,...,5,1,2,2,0,3,2,2,1,96
4,0,,10,30,124,113,0,1,1,1,...,14,1,2,2,0,3,2,2,2,1
